In [7]:
# import required libraries
import pandas as pd
import requests as re
import os

In [2]:
# read nyc_uber data
nyc_uber = pd.read_csv('nyc_uber_data.csv')

In [13]:
# instantiate authorization for yelp
apikey = os.environ['yelp_secret']
headers = {'Authorization': 'Bearer %s' % apikey}

In [3]:
nyc_uber.head()

,name,longitude,latitude,uber_count
0,Allerton,-73.859319,40.865788,860.0
1,Annadale,-74.178549,40.538114,11.0
2,Arden Heights,-74.185887,40.549286,10.0
3,Arlington,-74.165104,40.635325,0.0
4,Arrochar,-74.067124,40.596313,145.0


In [30]:
response = re.get('https://api.yelp.com/v3/businesses/search?location=NYC,Yorkville&term=food&price=1,2,3,4', headers=headers)

In [31]:
# testing response authorization
response.status_code

200

In [32]:
r = response.json()

In [42]:
# testing list creation from for loops
x = []
for i in r['businesses']:
    x.append(i['price'])
print(x)

['$$', '$$', '$$', '$$', '$$', '$$', '$', '$$', '$$', '$$', '$$', '$', '$$', '$', '$$', '$$', '$$', '$$', '$$', '$']


In [63]:
# creating function from testing for loops above
empdict = {}
def create_price():
    '''
    From an instantiated empty dictionary, creates a dictionary with NYC neighborhood names as keys and price as values
        Parameters:
            None
        Returns:
            None
    '''
    for i in nyc_uber['name']:
        emplist = []
        response = re.get(f'https://api.yelp.com/v3/businesses/search?location=NYC,{i}&term=food&price=1,2,3,4', headers=headers)
        r = response.json()
        for k in r['businesses']:
            emplist.append(k['price'])
        empdict[i] = emplist
    return

In [65]:
# run function to create empdict with NYC neighborhood names as keys and price as values
create_price()

In [67]:
# turn dictionary into dataframe
df_price = pd.DataFrame(empdict)

In [72]:
# replace $ signs from yelp data with numeric values
df_price1=df_price.replace({'$':1,
                 '$$':2,
                 '$$$':3,
                 '$$$$':4})

In [79]:
# transpose dataframe to turn NYC neighborhood names into indices
df_price2 = df_price1.T

In [82]:
# turn NYC neighborhood names from indices into a column
df_price2.reset_index(level=0, inplace=True)

In [89]:
# rename column from index to name to match nyc_uber data
df_price2.rename(columns={'index':'name'}, inplace=True)

In [98]:
# create new column as an average price of each neighborhood
df_price2['avg_price'] = df_price2.mean(axis=1)

In [102]:
df_price3 = df_price2[['name', 'avg_price']]

In [105]:
df_price3

,name,avg_price
0,Allerton,1.50
1,Annadale,1.75
2,Arden Heights,1.70
3,Arlington,1.55
4,Arrochar,1.75
...,...,...
297,Woodhaven,1.40
298,Woodlawn,1.40
299,Woodrow,1.75
300,Woodside,1.40


In [106]:
# merge price to nyc_uber data
nyc_up = nyc_uber.merge(df_price3, how='outer', on='name')

In [109]:
nyc_up.to_csv('nyc_uber_price.csv', index=False)